<div class='bar_title'></div>

*Decision Support Systems*

# Bonus 01

Toni Greif <br>

Winter Semester 19/20

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Implementation" data-toc-modified-id="Implementation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Implementation</a></span></li></ul></div>

In [1]:
!pip install pulp
from pulp import *
import pandas as pd
from operator import add

## Implementation

In [2]:
# Create model
m = LpProblem("Factory Planning", sense=pulp.LpMaximize)

In [3]:
# Dictionaries
profit = [10, 6, 8, 4, 11, 9, 3]

num_machines= {'Grinding': 4,
               'Drilling': 3,
               'Boring': 1,
               'Planing': 1}


production_time = {'Grinding': [0.5, 0.7, 0, 0, 0.3, 0.2, 0.5],
                   'Drilling': [0.1, 0.2, 0.8, 0.3, 0, 0.6, 0],
                   'Boring': [0.05, 0.03, 0, 0.07, 0.1, 0, 0.8],
                   'Planing': [0, 0, 0.01, 0, 0.05, 0, 0.05]}

demand = {1: [500, 1000, 300, 300, 800, 200, 100],
          2: [600, 500, 200, 0, 400, 300, 150],
          3: [300, 600, 0, 0, 500, 400, 100],
          4: [200, 300, 400, 500, 200, 0, 100],
          5: [0, 100, 500, 100, 1000, 300, 0],
          6: [500, 500, 100, 300, 1100, 500, 60]}

maintenance = {1: {'Grinding': 1, 'Drilling': 0, 'Boring': 0, 'Planing': 0},
               2: {'Grinding': 0, 'Drilling': 0, 'Boring': 0, 'Planing': 0},
               3: {'Grinding': 0, 'Drilling': 0, 'Boring': 1, 'Planing': 0},
               4: {'Grinding': 0, 'Drilling': 1, 'Boring': 0, 'Planing': 0},
               5: {'Grinding': 1, 'Drilling': 1, 'Boring': 0, 'Planing': 0},
               6: {'Grinding': 0, 'Drilling': 0, 'Boring': 0, 'Planing': 1}}

costs = {1: {'Grinding': 0.5, 'Drilling': 0.4, 'Boring': 0.4, 'Planing': 0.3},
         2: {'Grinding': 0.7, 'Drilling': 0.8, 'Boring': 0.7, 'Planing': 0.6},
         3: {'Grinding': 1.2, 'Drilling': 1.2, 'Boring': 1.2, 'Planing': 1.3},
         4: {'Grinding': 1, 'Drilling': 1.1, 'Boring': 1.1, 'Planing': 1.4},
         5: {'Grinding': 0.8, 'Drilling': 0.6, 'Boring': 0.4, 'Planing': 0.5},
         6: {'Grinding': 1, 'Drilling': 1.2, 'Boring': 1.8, 'Planing': 1}}

In [4]:
# Sets
products = range(len(profit))
machines = production_time.keys()
periods = demand.keys()

In [5]:
# Parameters
store_ub = 400
store_costs_per_unit = 0.05
final_storage = 0

days_per_period = 24
hours_per_shift = 8
shifts_per_day = 2

In [6]:
# Variables
make = LpVariable.dicts("make", (products, periods), lowBound=0, cat='Continuous')
sell = LpVariable.dicts("sell", (products, periods), lowBound=0, cat='Continuous')
store = LpVariable.dicts("store", (products, periods), lowBound=0, upBound=store_ub, cat='Continuous')

# Objective
m += lpSum([profit[pro] * sell[pro][per]
            - store_costs_per_unit * store[pro][per] for pro in products for per in periods])

In [7]:
# Constraints
for mach in machines:  
    for per in periods:
        m += lpSum([production_time[mach][pro] * make[pro][per] for pro in products]) <= hours_per_shift * shifts_per_day * days_per_period * (num_machines[mach] - maintenance[per][mach])
        
for pro in products:
    m += store[pro][6] == final_storage
     
for pro in products:
    for per in periods:
        m += sell[pro][per] <= demand[per][pro]

for pro in products:
    for per in periods:
        if per == 1:
            m += 0 + make[pro][1] == sell[pro][1] + store[pro][1]
        else:
            m += store[pro][per-1] + make[pro][per] == sell[pro][per] + store[pro][per]   

In [8]:
# Solve problem
m.solve()
print (LpStatus[m.status])
print (value(m.objective))

Optimal
105573.42858


In [9]:
out = [[pro, per, make[pro][per].varValue, sell[pro][per].varValue, store[pro][per].varValue]
       for pro in products for per in periods]
print (pd.DataFrame(out))
print(pd.DataFrame(out).sum())

    0  1           2           3      4
0   0  1   500.00000   500.00000    0.0
1   0  2   900.00000   600.00000  300.0
2   0  3     0.00000   300.00000    0.0
3   0  4   200.00000   200.00000    0.0
4   0  5     0.00000     0.00000    0.0
5   0  6   500.00000   500.00000    0.0
6   1  1   888.57143   888.57143    0.0
7   1  2   900.00000   500.00000  400.0
8   1  3     0.00000   400.00000    0.0
9   1  4   300.00000   300.00000    0.0
10  1  5   100.00000   100.00000    0.0
11  1  6   500.00000   500.00000    0.0
12  2  1   300.00000   300.00000    0.0
13  2  2   200.00000   200.00000    0.0
14  2  3     0.00000     0.00000    0.0
15  2  4   400.00000   400.00000    0.0
16  2  5   600.00000   500.00000  100.0
17  2  6     0.00000   100.00000    0.0
18  3  1   300.00000   300.00000    0.0
19  3  2     0.00000     0.00000    0.0
20  3  3     0.00000     0.00000    0.0
21  3  4   500.00000   500.00000    0.0
22  3  5   100.00000   100.00000    0.0
23  3  6   300.00000   300.00000    0.0


In [10]:
mean_costs = pd.DataFrame(production_time).sum(axis=1).tolist()
prices =  [sum(x) for x in zip(profit, mean_costs)]
print(mean_costs)
print(prices)

[0.65, 0.9299999999999999, 0.81, 0.37, 0.45, 0.8, 1.35]
[10.65, 6.93, 8.81, 4.37, 11.45, 9.8, 4.35]


In [11]:
lpSum([prices[pro] * sell[pro][per].varValue - store_costs_per_unit * store[pro][per].varValue 
       for pro in products for per in periods]
     +[production_time[mach][pro] * make[pro][per].varValue * -costs[per][mach]
       for pro in products for per in periods for mach in machines])

107638.15143784002

In [12]:
# Objective
m += lpSum([prices[pro] * sell[pro][per] - store_costs_per_unit * store[pro][per] 
       for pro in products for per in periods]
     +[production_time[mach][pro] * make[pro][per] * -costs[per][mach]
       for pro in products for per in periods for mach in machines])

C:\Users\Toni Greif\Anaconda3\lib\site-packages\pulp\pulp.py:1349: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [13]:
# Solve problem
m.solve()
print (LpStatus[m.status])
print (value(m.objective))

Optimal
108050.70143784002


In [14]:
out_ext = [[pro, per, make[pro][per].varValue, sell[pro][per].varValue, store[pro][per].varValue]
           for pro in products for per in periods]
print (pd.DataFrame(out_ext))
print(pd.DataFrame(out_ext).sum())

    0  1           2           3      4
0   0  1   500.00000   500.00000    0.0
1   0  2   900.00000   600.00000  300.0
2   0  3     0.00000   300.00000    0.0
3   0  4   200.00000   200.00000    0.0
4   0  5   400.00000     0.00000  400.0
5   0  6   100.00000   500.00000    0.0
6   1  1   888.57143   888.57143    0.0
7   1  2   900.00000   500.00000  400.0
8   1  3     0.00000   400.00000    0.0
9   1  4   300.00000   300.00000    0.0
10  1  5   500.00000   100.00000  400.0
11  1  6   100.00000   500.00000    0.0
12  2  1   700.00000   300.00000  400.0
13  2  2   200.00000   200.00000  400.0
14  2  3     0.00000     0.00000  400.0
15  2  4   400.00000   400.00000  400.0
16  2  5   200.00000   500.00000  100.0
17  2  6     0.00000   100.00000    0.0
18  3  1   700.00000   300.00000  400.0
19  3  2     0.00000     0.00000  400.0
20  3  3     0.00000     0.00000  400.0
21  3  4   100.00000   500.00000    0.0
22  3  5   400.00000   100.00000  300.0
23  3  6     0.00000   300.00000    0.0
